In [1]:
# importing all the needed libraries for the whole project.
import pandas as pd
import numpy as np
from datetime import datetime
import sidetable
import openai
import os
import re
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
import gradio as gr
from langchain.document_loaders import DataFrameLoader
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.embeddings import FakeEmbeddings
from langchain.vectorstores import FAISS
import gpt4all
from gpt4all import GPT4All
from langchain.document_loaders import PyPDFLoader
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pdf2image import convert_from_path
import logging

/Users/cesarchaparro/opt/anaconda3/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/Users/cesarchaparro/opt/anaconda3/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")


In [2]:
users = pd.read_csv('/Users/cesarchaparro/Desktop/TripleTen/Personal_projects/benedict/Users.csv')

In [3]:
users.head()

,_id,NAME
0,5922b097d6bf642200d2a38c,רון קורן
1,592309505497b2220052490e,אפרת ריבלין
2,59230abbd6bf642200d2a82a,אדיר שי
3,59230ae1d6bf642200d2a82d,נועה אולבסקי
4,59231036d6bf642200d2a86b,דניאל מנדלסון


In [4]:
benedict = pd.read_csv('/Users/cesarchaparro/Desktop/TripleTen/Personal_projects/benedict/Benedict_June_2023.csv')

/var/folders/qk/q0fv26q13m760q3ch0fljs240000gn/T/ipykernel_18061/196370269.py:1: DtypeWarning: Columns (13,15,16,17,18,19,20,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  benedict = pd.read_csv('/Users/cesarchaparro/Desktop/TripleTen/Personal_projects/benedict/Benedict_June_2023.csv')


In [5]:
benedict.head()

,tlogId,businessDate,opened,closed,loyalty,ownerId,Owner,source,orderType,tableseats,...,grossAmount,defaultPrice,refundAmountIncludeVat,refundAmountExcludeVat,refundVatAmount,operatorAmount,menuName,sectionName,sectionOfferName,price
0,6477bb6aeefee051a9ea604d,01/06/2023,01/06/2023 00:24:51,01/06/2023 00:26:02,0,6054e6be0e223adf6735db65,נועם צרפתי,restaurantOnPremise,seated,2.0,...,15.0,15.0,NaN,NaN,NaN,positiveSales,שתיה,שתיה חמה,הפוך גדול,15.0
1,6477bb6aeefee051a9ea604d,01/06/2023,01/06/2023 00:24:51,01/06/2023 00:26:02,0,6054e6be0e223adf6735db65,נועם צרפתי,restaurantOnPremise,seated,2.0,...,0.0,0.0,NaN,NaN,NaN,positiveSales,שתיה,שתיה חמה,הפוך גדול,15.0
2,6477bb6aeefee051a9ea604d,01/06/2023,01/06/2023 00:24:51,01/06/2023 00:26:02,0,6054e6be0e223adf6735db65,נועם צרפתי,restaurantOnPremise,seated,2.0,...,15.0,15.0,NaN,NaN,NaN,positiveSales,שתיה,שתיה חמה,הפוך גדול,15.0
3,6477bb6aeefee051a9ea604d,01/06/2023,01/06/2023 00:24:51,01/06/2023 00:26:02,0,6054e6be0e223adf6735db65,נועם צרפתי,restaurantOnPremise,seated,2.0,...,0.0,0.0,NaN,NaN,NaN,positiveSales,שתיה,שתיה חמה,הפוך גדול,15.0
4,6477bd8d1c901f25455a78bb,01/06/2023,31/05/2023 22:39:11,01/06/2023 00:35:09,0,6367778a025a3bb0f595df4f,אור אלי,restaurantOnPremise,seated,4.0,...,41.0,41.0,NaN,NaN,NaN,positiveSales,אוכל,פנקייקס,מיני קינדר,41.0


In [6]:
benedict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112181 entries, 0 to 112180
Data columns (total 52 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   tlogId                  112181 non-null  object 
 1   businessDate            112181 non-null  object 
 2   opened                  112181 non-null  object 
 3   closed                  112181 non-null  object 
 4   loyalty                 112181 non-null  int64  
 5   ownerId                 112181 non-null  object 
 6   Owner                   112181 non-null  object 
 7   source                  112181 non-null  object 
 8   orderType               112181 non-null  object 
 9   tableseats              107358 non-null  float64
 10  vatPercent              112181 non-null  int64  
 11  workflowProfileName     109311 non-null  object 
 12  workflowProfileType     109311 non-null  object 
 13  ordererRegionName       174 non-null     object 
 14  mainTableTag        

In [7]:
# getting info and memory usage for dataframe.
benedict.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112181 entries, 0 to 112180
Data columns (total 52 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   tlogId                  112181 non-null  object 
 1   businessDate            112181 non-null  object 
 2   opened                  112181 non-null  object 
 3   closed                  112181 non-null  object 
 4   loyalty                 112181 non-null  int64  
 5   ownerId                 112181 non-null  object 
 6   Owner                   112181 non-null  object 
 7   source                  112181 non-null  object 
 8   orderType               112181 non-null  object 
 9   tableseats              107358 non-null  float64
 10  vatPercent              112181 non-null  int64  
 11  workflowProfileName     109311 non-null  object 
 12  workflowProfileType     109311 non-null  object 
 13  ordererRegionName       174 non-null     object 
 14  mainTableTag        

***

In [8]:
# Define the mapping of old column names to new column names
column_mapping = {
    "tlogId": "transaction_id",
    "businessDate": "transaction_date",
    "opened": "order_open_time",
    "closed": "order_close_time",
    "loyalty": "is_loyalty_member",
    "ownerId": "employee_id",
    "Owner": "employee_name",
    "source": "order_source",
    "orderType": "order_type",
    "tableseats": "number_of_diners",
    "vatPercent": "vat_percentage",
    "workflowProfileName": "workflow_profile_name",
    "workflowProfileType": "workflow_profile_type",
    "ordererRegionName": "customer_region",
    "mainTableTag": "restaurant_table_area",
    "itemId": "item_id",
    "orderedItemId": "ordered_item_id",
    "type": "item_type",
    "orderedOfferId": "ordered_offer_id",
    "itemName": "item_name",
    "offerName": "offer_name",
    "isModifier": "is_item_modifier",
    "chairNumber": "chair_number",
    "prepared": "is_prepared",
    "return": "is_returned",
    "cancelled": "is_cancelled",
    "sold": "is_sold",
    "netAmount": "net_amount",
    "refundAmount": "refund_amount",
    "generalAmount": "total_amount",
    "netAmountVat": "vat_amount",
    "returnAmount": "returned_amount",
    "cancellationAmount": "cancelled_amount",
    "priceReduction": "discount_amount",
    "isNegativeValue": "is_credit",
    "modifierSold": "modifiers_sold",
    "modifierPrepared": "modifiers_prepared",
    "netAmountIncludeVat": "net_amount_with_vat",
    "netAmountExcludeVat": "net_amount_without_vat",
    "salesAmountIncludeVat": "total_sales_with_vat",
    "salesAmountExcludeVat": "total_sales_without_vat",
    "salesVatAmount": "sales_vat_amount",
    "grossAmount": "gross_amount",
    "defaultPrice": "base_price",
    "refundAmountIncludeVat": "refund_with_vat",
    "refundAmountExcludeVat": "refund_without_vat",
    "refundVatAmount": "refund_vat_amount",
    "operatorAmount": "operator_amount",
    "menuName": "menu_section",
    "sectionName": "menu_subcategory",
    "sectionOfferName": "menu_secondary_subcategory",
    "price": "final_item_price"
}

# Rename the columns
benedict.rename(columns=column_mapping, inplace=True)

# Display the updated column names (Optional)
print(benedict.columns)

Index(['transaction_id', 'transaction_date', 'order_open_time',
       'order_close_time', 'is_loyalty_member', 'employee_id', 'employee_name',
       'order_source', 'order_type', 'number_of_diners', 'vat_percentage',
       'workflow_profile_name', 'workflow_profile_type', 'customer_region',
       'restaurant_table_area', 'item_id', 'ordered_item_id', 'item_type',
       'ordered_offer_id', 'item_name', 'offer_name', 'is_item_modifier',
       'chair_number', 'is_prepared', 'is_returned', 'is_cancelled', 'is_sold',
       'net_amount', 'refund_amount', 'total_amount', 'vat_amount',
       'returned_amount', 'cancelled_amount', 'discount_amount', 'is_credit',
       'modifiers_sold', 'modifiers_prepared', 'net_amount_with_vat',
       'net_amount_without_vat', 'total_sales_with_vat',
       'total_sales_without_vat', 'sales_vat_amount', 'gross_amount',
       'base_price', 'refund_with_vat', 'refund_without_vat',
       'refund_vat_amount', 'operator_amount', 'menu_section',
       '

In [9]:
benedict['item_id'].head()

0    5c5c152d8906156c08681fe6
1    5fe19a5c381027c4c3d70b56
2    5c5c152d8906156c08681fe6
3    5fe19a5c381027c4c3d70b56
4    5c5c15568906156c086822c3
Name: item_id, dtype: object

In [10]:
benedict['ordered_item_id'].head()

0    5c5c15c78906156c086828ad
1    5c5c15c78906156c086828ad
2    5c5c15c78906156c086828ad
3    5c5c15c78906156c086828ad
4    619b9faf82d9b4febd5b2dd2
Name: ordered_item_id, dtype: object

In [11]:
# checking for missing values on costs data set.
benedict.stb.missing(style = True)

,missing,total,percent
refund_vat_amount,"112,166","112,181",99.99%
refund_with_vat,"112,166","112,181",99.99%
refund_without_vat,"112,166","112,181",99.99%
customer_region,"112,007","112,181",99.84%
ordered_offer_id,"48,152","112,181",42.92%
final_item_price,"47,107","112,181",41.99%
menu_subcategory,"47,107","112,181",41.99%
menu_section,"47,107","112,181",41.99%
menu_secondary_subcategory,"47,107","112,181",41.99%
offer_name,"47,083","112,181",41.97%


***

In [12]:
unique_values = benedict['refund_with_vat'].unique()
print("Unique values in 'refund_with_vat':", unique_values)

Unique values in 'refund_with_vat': [ nan  26.  80.  15.  45.  32. 213. 349.  18.  89.  16.  63.  13. 145.
 210.]


In [13]:
benedict['refund_with_vat'] = benedict['refund_with_vat'].fillna(0.0)

In [14]:
unique_values = benedict['number_of_diners'].unique()
print("Unique values in 'number_of_diners':", unique_values)

Unique values in 'number_of_diners': [ 2.  4.  5.  3.  7. 10. nan  6.  1.]


In [15]:
benedict['number_of_diners'] = benedict['number_of_diners'].fillna(0.0)

In [16]:
unique_values = benedict['chair_number'].unique()
print("Unique values in 'chair_number':", unique_values)

Unique values in 'chair_number': [ 1.  2.  0.  3.  4.  8.  5.  6.  9.  7. nan]


In [17]:
# Calculate the mean, round it to the nearest whole number, convert to int, and fill NaN values
mean_value = int(round(benedict['chair_number'].mean()))
benedict['chair_number'] = benedict['chair_number'].fillna(mean_value)

# Display the updated unique values
unique_values = benedict['chair_number'].unique()
print("Unique values in 'chair_number' after filling NaN:", unique_values)

Unique values in 'chair_number' after filling NaN: [1. 2. 0. 3. 4. 8. 5. 6. 9. 7.]


In [18]:
unique_values = benedict['net_amount'].unique()
print("Unique values in 'net_amount':", unique_values)

Unique values in 'net_amount': [15.    0.   41.   ... 62.66 15.26   nan]


In [19]:
benedict['net_amount'].describe()

count    65125.000000
mean        12.882088
std         24.690057
min       -349.000000
25%          0.000000
50%          0.000000
75%         14.000000
max       2000.000000
Name: net_amount, dtype: float64

In [20]:
# Fill NaN values with the median
median_value = benedict['net_amount'].median()
benedict['net_amount'] = benedict['net_amount'].fillna(median_value)

In [21]:
unique_values = benedict['net_amount'].unique()
print("Unique values in 'net_amount':", unique_values)

Unique values in 'net_amount': [15.    0.   41.   ... 14.77 62.66 15.26]


In [22]:
unique_values = benedict['refund_amount'].unique()
print("Unique values in 'refund_amount':", unique_values)

Unique values in 'refund_amount': [ 0. nan]


In [23]:
benedict['refund_amount'].describe()

count    65125.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: refund_amount, dtype: float64

In [24]:
# Fill NaN values with the median
median_value = benedict['refund_amount'].median()
benedict['refund_amount'] = benedict['refund_amount'].fillna(median_value)

In [25]:
unique_values = benedict['refund_amount'].unique()
print("Unique values in 'refund_amount':", unique_values)

Unique values in 'refund_amount': [0.]


In [26]:
unique_values = benedict['total_amount'].unique()
print("Unique values in 'total_amount':", unique_values)

Unique values in 'total_amount': [   0.  -26. 2000.  -80.  -15.  -45.   12.  -32. -213. -349.  400.  -18.
  -89.  -16.  -63.  -13. -145. -210.  135.   nan]


In [27]:
benedict['total_amount'].describe()

count    65125.000000
mean         0.018441
std          8.252003
min       -349.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       2000.000000
Name: total_amount, dtype: float64

In [28]:
benedict['total_amount'] = benedict['total_amount'].fillna(0.0)

In [29]:
unique_values = benedict['total_amount'].unique()
print("Unique values in 'total_amount':", unique_values)

Unique values in 'total_amount': [   0.  -26. 2000.  -80.  -15.  -45.   12.  -32. -213. -349.  400.  -18.
  -89.  -16.  -63.  -13. -145. -210.  135.]


In [30]:
unique_values = benedict['vat_amount'].unique()
print("Unique values in 'vat_amount':", unique_values)

Unique values in 'vat_amount': [ 2.180e+00  0.000e+00  5.960e+00  3.350e+00  1.310e+00  1.003e+01
  9.150e+00  9.880e+00  4.650e+00  4.660e+00  6.250e+00  2.320e+00
  8.700e-01  2.170e+00  1.450e+00  2.760e+00  2.620e+00  1.148e+01
  1.900e+00  1.264e+01  1.890e+00  1.020e+00  6.680e+00  7.300e-01
  8.570e+00  8.430e+00  1.740e+00  3.050e+00  2.030e+00  2.040e+00
  5.520e+00  5.530e+00  6.540e+00  6.390e+00  3.200e+00  8.860e+00
  3.490e+00  5.670e+00  8.280e+00  5.660e+00  3.480e+00  1.160e+00
  1.424e+01  1.061e+01  5.200e+00  7.600e-01  1.750e+00  5.090e+00
  1.250e+01  7.410e+00  2.830e+00  1.880e+00  4.940e+00  4.080e+00
  9.360e+00  9.740e+00  5.950e+00  6.970e+00  2.770e+00  1.600e+00
  1.032e+01  4.070e+00  2.106e+01  2.200e+00  2.610e+00  4.930e+00
  1.133e+01  9.000e-01  1.002e+01  1.031e+01  5.680e+00  4.210e+00
  4.500e+00  3.950e+00  2.330e+00  5.050e+00  9.010e+00  8.140e+00
  1.365e+01  3.460e+00  5.230e+00  7.850e+00  1.075e+01  8.420e+00
  5.800e-01  5.240e+00  1.480e+

In [31]:
benedict['vat_amount'].describe()

count    65125.000000
mean         1.871759
std          3.587591
min        -50.710000
25%          0.000000
50%          0.000000
75%          2.030000
max        290.600000
Name: vat_amount, dtype: float64

In [32]:
benedict['vat_amount'] = benedict['vat_amount'].fillna(0.0)

In [33]:
unique_values = benedict['returned_amount'].unique()
print("Unique values in 'returned_amount':", unique_values)

Unique values in 'returned_amount': [ 0. 58. 67. 12. 68. 11. 19. 13. 41. 59. 21. 16. 78. 24. 39.  4.  6. 18.
 71. 63.  7. 32. 75. 22. 35. 72. 23. 45. 38. 15. 74. 54. 53. 29. 31.  9.
  5. 69. 73. 14. 79. 62.  8. 10. 65. 51. 44. 43. nan]


In [34]:
benedict['returned_amount'].describe()

count    65125.000000
mean         0.226242
std          3.337777
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         79.000000
Name: returned_amount, dtype: float64

In [35]:
benedict['returned_amount'] = benedict['returned_amount'].fillna(0.0)

In [36]:
unique_values = benedict['returned_amount'].unique()
print("Unique values in 'returned_amount':", unique_values)

Unique values in 'returned_amount': [ 0. 58. 67. 12. 68. 11. 19. 13. 41. 59. 21. 16. 78. 24. 39.  4.  6. 18.
 71. 63.  7. 32. 75. 22. 35. 72. 23. 45. 38. 15. 74. 54. 53. 29. 31.  9.
  5. 69. 73. 14. 79. 62.  8. 10. 65. 51. 44. 43.]


In [37]:
unique_values = benedict['cancelled_amount'].unique()
print("Unique values in 'cancelled_amount':", unique_values)

Unique values in 'cancelled_amount': [  0.  32.  41.   6.  19.  59.  24.  13.  11.  61.  73.  44.  39.  26.
  69.  16.  43.  68.  15.  74.  67.  12.  10.  29.  78.  71.  21.  75.
  18.   7.  63.  38.  58.  86.  22.  53.  28.  23.  98.  79.  45.  31.
   8.   5. 100.  72.   9.  46.  48. 108.  34.  14.  54.  27.  35. 259.
  49. 110.  nan]


In [ ]:
benedict['cancelled_amount'].describe()

count    65125.000000
mean         0.277559
std          3.615259
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        259.000000
Name: cancelled_amount, dtype: float64

In [39]:
benedict['cancelled_amount'] = benedict['cancelled_amount'].fillna(0.0)

In [40]:
unique_values = benedict['cancelled_amount'].unique()
print("Unique values in 'cancelled_amount':", unique_values)

Unique values in 'cancelled_amount': [  0.  32.  41.   6.  19.  59.  24.  13.  11.  61.  73.  44.  39.  26.
  69.  16.  43.  68.  15.  74.  67.  12.  10.  29.  78.  71.  21.  75.
  18.   7.  63.  38.  58.  86.  22.  53.  28.  23.  98.  79.  45.  31.
   8.   5. 100.  72.   9.  46.  48. 108.  34.  14.  54.  27.  35. 259.
  49. 110.]


In [41]:
unique_values = benedict['discount_amount'].unique()
print("Unique values in 'discount_amount':", unique_values)

Unique values in 'discount_amount': [0.000e+00 1.500e+01 3.200e+01 1.300e+01 3.800e+01 5.230e+00 7.700e-01
 2.149e+01 3.093e+01 3.580e+00 1.400e+01 5.800e+01 5.770e+00 6.700e+01
 1.200e+01 6.800e+01 8.400e-01 4.100e+01 6.810e+00 1.100e+01 1.900e+01
 6.000e+00 4.820e+00 6.180e+00 1.101e+01 5.900e+01 6.050e+00 4.220e+00
 1.960e+00 2.400e+01 1.810e+00 6.100e+01 2.100e+01 9.820e+00 2.180e+00
 7.170e+00 7.300e+01 5.000e+00 8.300e-01 4.400e+01 9.000e+00 1.800e+01
 1.318e+01 1.728e+01 7.030e+00 3.900e+01 3.520e+00 2.600e+01 2.548e+01
 6.900e+01 3.810e+00 1.600e+01 1.223e+01 1.713e+01 3.590e+00 4.350e+00
 1.470e+00 9.200e-01 3.530e+00 5.420e+00 4.080e+00 4.180e+00 1.060e+01
 6.250e+00 1.200e+00 1.190e+00 7.900e-01 1.930e+00 1.320e+00 2.880e+00
 1.510e+00 6.600e-01 8.500e-01 2.162e+01 7.600e+00 2.630e+00 1.315e+01
 1.690e+00 1.220e+01 2.441e+01 9.920e+00 5.949e+01 2.898e+01 4.300e+01
 7.690e+00 4.060e+00 1.250e+00 7.800e+01 4.830e+00 1.267e+01 1.967e+01
 8.910e+00 8.380e+00 8.480e+00 2.420e+00 

In [42]:
benedict['discount_amount'].describe()

count    65125.000000
mean         1.042887
std          6.324451
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        259.000000
Name: discount_amount, dtype: float64

In [43]:
benedict['discount_amount'] = benedict['discount_amount'].fillna(0.0)

In [44]:
unique_values = benedict['discount_amount'].unique()
print("Unique values in 'discount_amount':", unique_values)

Unique values in 'discount_amount': [0.000e+00 1.500e+01 3.200e+01 1.300e+01 3.800e+01 5.230e+00 7.700e-01
 2.149e+01 3.093e+01 3.580e+00 1.400e+01 5.800e+01 5.770e+00 6.700e+01
 1.200e+01 6.800e+01 8.400e-01 4.100e+01 6.810e+00 1.100e+01 1.900e+01
 6.000e+00 4.820e+00 6.180e+00 1.101e+01 5.900e+01 6.050e+00 4.220e+00
 1.960e+00 2.400e+01 1.810e+00 6.100e+01 2.100e+01 9.820e+00 2.180e+00
 7.170e+00 7.300e+01 5.000e+00 8.300e-01 4.400e+01 9.000e+00 1.800e+01
 1.318e+01 1.728e+01 7.030e+00 3.900e+01 3.520e+00 2.600e+01 2.548e+01
 6.900e+01 3.810e+00 1.600e+01 1.223e+01 1.713e+01 3.590e+00 4.350e+00
 1.470e+00 9.200e-01 3.530e+00 5.420e+00 4.080e+00 4.180e+00 1.060e+01
 6.250e+00 1.200e+00 1.190e+00 7.900e-01 1.930e+00 1.320e+00 2.880e+00
 1.510e+00 6.600e-01 8.500e-01 2.162e+01 7.600e+00 2.630e+00 1.315e+01
 1.690e+00 1.220e+01 2.441e+01 9.920e+00 5.949e+01 2.898e+01 4.300e+01
 7.690e+00 4.060e+00 1.250e+00 7.800e+01 4.830e+00 1.267e+01 1.967e+01
 8.910e+00 8.380e+00 8.480e+00 2.420e+00 

In [45]:
unique_values = benedict['net_amount_with_vat'].unique()
print("Unique values in 'net_amount_with_vat':", unique_values)

Unique values in 'net_amount_with_vat': [15.    0.   41.   ... 62.66 15.26   nan]


In [46]:
# Fill NaN values with the median
median_value = benedict['net_amount_with_vat'].median()
benedict['net_amount_with_vat'] = benedict['net_amount_with_vat'].fillna(median_value)

In [ ]:
unique_values = benedict['net_amount_with_vat'].unique()
print("Unique values in 'net_amount_with_vat':", unique_values)

Unique values in 'net_amount_with_vat': [15.    0.   41.   ... 14.77 62.66 15.26]


In [48]:
unique_values = benedict['net_amount_without_vat'].unique()
print("Unique values in 'net_amount_without_vat':", unique_values)

Unique values in 'net_amount_without_vat': [12.82  0.   35.04 ... 53.56 13.04   nan]


In [49]:
# Fill NaN values with the median
median_value = benedict['net_amount_without_vat'].median()
benedict['net_amount_without_vat'] = benedict['net_amount_without_vat'].fillna(median_value)

In [50]:
unique_values = benedict['net_amount_without_vat'].unique()
print("Unique values in 'net_amount_without_vat':", unique_values)

Unique values in 'net_amount_without_vat': [12.82  0.   35.04 ... 12.62 53.56 13.04]


In [51]:
unique_values = benedict['sales_vat_amount'].unique()
print("Unique values in 'sales_vat_amount':", unique_values)

Unique values in 'sales_vat_amount': [ 2.180e+00  0.000e+00  5.960e+00  3.350e+00  1.310e+00  1.003e+01
  9.150e+00  9.880e+00  4.650e+00  4.660e+00  6.250e+00  2.320e+00
  8.700e-01  2.170e+00  1.450e+00  2.760e+00  2.620e+00  1.148e+01
  1.900e+00  1.264e+01  1.890e+00  1.020e+00  6.680e+00  7.300e-01
  8.570e+00  8.430e+00  1.740e+00  3.050e+00  2.030e+00  2.040e+00
  5.520e+00  5.530e+00  6.540e+00  6.390e+00  3.200e+00  8.860e+00
  3.490e+00  5.670e+00  8.280e+00  5.660e+00  3.480e+00  1.160e+00
  1.424e+01  1.061e+01  5.200e+00  7.600e-01  1.750e+00  5.090e+00
  1.250e+01  7.410e+00  2.830e+00  1.880e+00  4.940e+00  4.080e+00
  9.360e+00  9.740e+00  5.950e+00  6.970e+00  2.770e+00  1.600e+00
  1.032e+01  4.070e+00  2.106e+01  2.200e+00  2.610e+00  4.930e+00
  1.133e+01  9.000e-01  1.002e+01  1.031e+01  5.680e+00  4.210e+00
  4.500e+00  3.950e+00  2.330e+00  5.050e+00  9.010e+00  8.140e+00
  1.365e+01  3.460e+00  5.230e+00  7.850e+00  1.075e+01  8.420e+00
  5.800e-01  5.240e+00  1

In [52]:
# Fill NaN values with the median
median_value = benedict['sales_vat_amount'].median()
benedict['sales_vat_amount'] = benedict['sales_vat_amount'].fillna(median_value)

In [53]:
unique_values = benedict['sales_vat_amount'].unique()
print("Unique values in 'sales_vat_amount':", unique_values)

Unique values in 'sales_vat_amount': [ 2.180e+00  0.000e+00  5.960e+00  3.350e+00  1.310e+00  1.003e+01
  9.150e+00  9.880e+00  4.650e+00  4.660e+00  6.250e+00  2.320e+00
  8.700e-01  2.170e+00  1.450e+00  2.760e+00  2.620e+00  1.148e+01
  1.900e+00  1.264e+01  1.890e+00  1.020e+00  6.680e+00  7.300e-01
  8.570e+00  8.430e+00  1.740e+00  3.050e+00  2.030e+00  2.040e+00
  5.520e+00  5.530e+00  6.540e+00  6.390e+00  3.200e+00  8.860e+00
  3.490e+00  5.670e+00  8.280e+00  5.660e+00  3.480e+00  1.160e+00
  1.424e+01  1.061e+01  5.200e+00  7.600e-01  1.750e+00  5.090e+00
  1.250e+01  7.410e+00  2.830e+00  1.880e+00  4.940e+00  4.080e+00
  9.360e+00  9.740e+00  5.950e+00  6.970e+00  2.770e+00  1.600e+00
  1.032e+01  4.070e+00  2.106e+01  2.200e+00  2.610e+00  4.930e+00
  1.133e+01  9.000e-01  1.002e+01  1.031e+01  5.680e+00  4.210e+00
  4.500e+00  3.950e+00  2.330e+00  5.050e+00  9.010e+00  8.140e+00
  1.365e+01  3.460e+00  5.230e+00  7.850e+00  1.075e+01  8.420e+00
  5.800e-01  5.240e+00  1

In [54]:
unique_values = benedict['gross_amount'].unique()
print("Unique values in 'gross_amount':", unique_values)

Unique values in 'gross_amount': [ 1.50e+01  0.00e+00  4.10e+01  2.30e+01  9.00e+00  6.90e+01  6.30e+01
  6.80e+01  3.20e+01  4.30e+01  1.60e+01  6.00e+00  1.00e+01  1.90e+01
  1.80e+01  7.90e+01  1.30e+01  8.70e+01  7.00e+00  4.60e+01  5.00e+00
  5.90e+01  5.80e+01  1.20e+01  2.10e+01  1.40e+01  3.80e+01  4.50e+01
  4.40e+01  2.20e+01  6.10e+01  2.40e+01  3.90e+01  5.70e+01  8.00e+00
  9.80e+01  7.30e+01  3.50e+01  8.60e+01  5.10e+01  3.40e+01  6.70e+01
  4.80e+01  1.10e+01  7.10e+01  2.80e+01  1.45e+02  7.80e+01  2.90e+01
  3.10e+01  5.60e+01  1.00e+02  3.60e+01  5.40e+01  7.40e+01  4.00e+00
  6.20e+01  1.70e+01  5.30e+01  2.00e+00  2.99e+02  2.69e+02  1.00e+00
  2.60e+01  2.59e+02  6.50e+01 -2.60e+01  7.20e+01  7.50e+01  1.05e+02
  1.10e+02  1.02e+02  2.00e+03 -8.00e+01 -1.50e+01  1.55e+02  4.90e+01
 -4.50e+01 -3.20e+01 -2.13e+02 -3.49e+02  4.00e+02  1.08e+02 -1.80e+01
  2.70e+01  9.90e+01 -8.90e+01  2.00e+01 -1.60e+01 -6.30e+01 -1.30e+01
 -1.45e+02 -2.10e+02  1.35e+02       nan]


In [55]:
# Fill NaN values with the median
median_value = benedict['gross_amount'].median()
benedict['gross_amount'] = benedict['gross_amount'].fillna(median_value)

In [56]:
unique_values = benedict['gross_amount'].unique()
print("Unique values in 'gross_amount':", unique_values)

Unique values in 'gross_amount': [ 1.50e+01  0.00e+00  4.10e+01  2.30e+01  9.00e+00  6.90e+01  6.30e+01
  6.80e+01  3.20e+01  4.30e+01  1.60e+01  6.00e+00  1.00e+01  1.90e+01
  1.80e+01  7.90e+01  1.30e+01  8.70e+01  7.00e+00  4.60e+01  5.00e+00
  5.90e+01  5.80e+01  1.20e+01  2.10e+01  1.40e+01  3.80e+01  4.50e+01
  4.40e+01  2.20e+01  6.10e+01  2.40e+01  3.90e+01  5.70e+01  8.00e+00
  9.80e+01  7.30e+01  3.50e+01  8.60e+01  5.10e+01  3.40e+01  6.70e+01
  4.80e+01  1.10e+01  7.10e+01  2.80e+01  1.45e+02  7.80e+01  2.90e+01
  3.10e+01  5.60e+01  1.00e+02  3.60e+01  5.40e+01  7.40e+01  4.00e+00
  6.20e+01  1.70e+01  5.30e+01  2.00e+00  2.99e+02  2.69e+02  1.00e+00
  2.60e+01  2.59e+02  6.50e+01 -2.60e+01  7.20e+01  7.50e+01  1.05e+02
  1.10e+02  1.02e+02  2.00e+03 -8.00e+01 -1.50e+01  1.55e+02  4.90e+01
 -4.50e+01 -3.20e+01 -2.13e+02 -3.49e+02  4.00e+02  1.08e+02 -1.80e+01
  2.70e+01  9.90e+01 -8.90e+01  2.00e+01 -1.60e+01 -6.30e+01 -1.30e+01
 -1.45e+02 -2.10e+02  1.35e+02]


In [57]:
unique_values = benedict['base_price'].unique()
print("Unique values in 'base_price':", unique_values)

Unique values in 'base_price': [ 15.    0.   41.   23.    9.   69.   63.    5.   68.   32.   43.    4.9
  16.    6.   10.   19.   18.   79.   13.   87.    7.   46.   59.   58.
  21.   14.   38.   45.   12.   44.   22.   61.   24.   39.    8.   98.
  73.   35.   86.   62.   34.   71.   67.   56.   48.   11.   28.  145.
  78.   29.   31.   10.9  51.  100.   36.   54.   74.    4.   17.   53.
  26.    1.    2.  299.    3.  279.   12.9 259.   65.   72.   75.  105.
 110.  102.  155.   49.  108.   27.   99.   20.    nan]


In [58]:
# Fill NaN values with the median
median_value = benedict['base_price'].median()
benedict['base_price'] = benedict['base_price'].fillna(median_value)

In [59]:
unique_values = benedict['base_price'].unique()
print("Unique values in 'base_price':", unique_values)

Unique values in 'base_price': [ 15.    0.   41.   23.    9.   69.   63.    5.   68.   32.   43.    4.9
  16.    6.   10.   19.   18.   79.   13.   87.    7.   46.   59.   58.
  21.   14.   38.   45.   12.   44.   22.   61.   24.   39.    8.   98.
  73.   35.   86.   62.   34.   71.   67.   56.   48.   11.   28.  145.
  78.   29.   31.   10.9  51.  100.   36.   54.   74.    4.   17.   53.
  26.    1.    2.  299.    3.  279.   12.9 259.   65.   72.   75.  105.
 110.  102.  155.   49.  108.   27.   99.   20. ]


In [60]:
unique_values = benedict['refund_without_vat'].unique()
print("Unique values in 'refund_without_vat':", unique_values)

Unique values in 'refund_without_vat': [   nan  22.22  68.38  12.82  38.46  27.35 182.05 298.29  15.38  76.07
  13.68  53.85  11.11 123.93 179.49]


In [61]:
benedict['refund_without_vat'].describe()

count     15.000000
mean      76.695333
std       84.093709
min       11.110000
25%       18.800000
50%       38.460000
75%      100.000000
max      298.290000
Name: refund_without_vat, dtype: float64

In [62]:
# Fill NaN values with the median
median_value = benedict['refund_without_vat'].median()
benedict['refund_without_vat'] = benedict['refund_without_vat'].fillna(median_value)

In [63]:
unique_values = benedict['refund_without_vat'].unique()
print("Unique values in 'refund_without_vat':", unique_values)

Unique values in 'refund_without_vat': [ 38.46  22.22  68.38  12.82  27.35 182.05 298.29  15.38  76.07  13.68
  53.85  11.11 123.93 179.49]


In [64]:
unique_values = benedict['refund_vat_amount'].unique()
print("Unique values in 'refund_vat_amount':", unique_values)

Unique values in 'refund_vat_amount': [  nan  3.78 11.62  2.18  6.54  4.65 30.95 50.71  2.62 12.93  2.32  9.15
  1.89 21.07 30.51]


In [ ]:
benedict['refund_vat_amount'].describe()

count    15.000000
mean     13.038000
std      14.295799
min       1.890000
25%       3.200000
50%       6.540000
75%      17.000000
max      50.710000
Name: refund_vat_amount, dtype: float64

In [66]:
# Fill NaN values with the median
median_value = benedict['refund_vat_amount'].median()
benedict['refund_vat_amount'] = benedict['refund_vat_amount'].fillna(median_value)

In [67]:
unique_values = benedict['refund_vat_amount'].unique()
print("Unique values in 'refund_vat_amount':", unique_values)

Unique values in 'refund_vat_amount': [ 6.54  3.78 11.62  2.18  4.65 30.95 50.71  2.62 12.93  2.32  9.15  1.89
 21.07 30.51]


In [68]:
unique_values = benedict['operator_amount'].unique()
print("Unique values in 'operator_amount':", unique_values)

Unique values in 'operator_amount': ['positiveSales' 'negativeSales' nan]


In [69]:
benedict['operator_amount'] = benedict['operator_amount'].fillna('unknown')

In [70]:
benedict['operator_amount'].unique()

array(['positiveSales', 'negativeSales', 'unknown'], dtype=object)

In [71]:
benedict['operator_amount'] = benedict['operator_amount'].astype('category')

In [72]:
unique_values = benedict['final_item_price'].unique()
print("Unique values in 'final_item_price':", unique_values)

Unique values in 'final_item_price': [ 15.  41.   0.  23.   9.  69.  63.  68.  32.  43.  16.   6.  10.  19.
  18.  79.  13.  87.   7.  46.  59.  58.  21.  14.  38.  45.  12.  44.
  22.  61.  24.  39.  57.  98.  73.  35.  86.  51.  34.  67.  48.  11.
  71.  28. 145.  78.  29.  31.  74.  56. 100.  36.  54.  53.   5.  26.
   2. 299. 279.  72.   8. 259.  65.  nan  75.   1. 105. 110. 102.   4.
 155.  49.  17. 108.  27.  99.  20.]


In [73]:
# Fill NaN values with the median
median_value = benedict['final_item_price'].median()
benedict['final_item_price'] = benedict['final_item_price'].fillna(median_value)

In [74]:
unique_values = benedict['final_item_price'].unique()
print("Unique values in 'final_item_price':", unique_values)

Unique values in 'final_item_price': [ 15.  41.   0.  23.   9.  69.  63.  68.  32.  43.  16.   6.  10.  19.
  18.  79.  13.  87.   7.  46.  59.  58.  21.  14.  38.  45.  12.  44.
  22.  61.  24.  39.  57.  98.  73.  35.  86.  51.  34.  67.  48.  11.
  71.  28. 145.  78.  29.  31.  74.  56. 100.  36.  54.  53.   5.  26.
   2. 299. 279.  72.   8. 259.  65.  75.   1. 105. 110. 102.   4. 155.
  49.  17. 108.  27.  99.  20.]


In [75]:
benedict['customer_region'].unique()

array([nan, 'הרצליה עיר', 'פיתוח וכפ"ש', 'רעננה'], dtype=object)

In [76]:
benedict['customer_region'] = benedict['customer_region'].fillna('none')

In [77]:
benedict['customer_region'].unique()

array(['none', 'הרצליה עיר', 'פיתוח וכפ"ש', 'רעננה'], dtype=object)

In [78]:
benedict['customer_region'] = benedict['customer_region'].astype('category')

In [79]:
benedict['workflow_profile_name'].unique()

array(['בר', 'בפנים', 'עמדה צפונית', 'עמדה דרומית', 'חוץ חדש', nan,
       'TA | משלוחים', 'עמדת בר', 'עמדה בייקרי'], dtype=object)

In [80]:
benedict['workflow_profile_name'] = benedict['workflow_profile_name'].fillna('none')

In [81]:
benedict['workflow_profile_name'] = benedict['workflow_profile_name'].astype('category')

In [82]:
benedict['workflow_profile_type'].unique()

array(['WProfile', 'OTCProfile', nan, 'TDProfile', 'BProfile'],
      dtype=object)

In [83]:
benedict['workflow_profile_type'] = benedict['workflow_profile_type'].fillna('none')

In [84]:
benedict['workflow_profile_type'] = benedict['workflow_profile_type'].astype('category')

In [85]:
benedict['restaurant_table_area'].unique()

array(['bar', 'restaurant', nan], dtype=object)

In [86]:
benedict['restaurant_table_area'] = benedict['restaurant_table_area'].fillna('none')

In [87]:
benedict['restaurant_table_area'] = benedict['restaurant_table_area'].astype('category')

In [88]:
benedict['item_id'].unique()

array(['5c5c152d8906156c08681fe6', '5fe19a5c381027c4c3d70b56',
       '5c5c15568906156c086822c3', '5db7dbacd5a6a1db8db55363',
       '5c5c154f8906156c08682268', '5eae8379af1e062a369baf72',
       '62039ee42be1c81b82a45bd5', '5c5c151b8906156c08681e3e',
       '5c5c15468906156c086821cc', '5fe19aa91f4e8970d0f915e3',
       '5dafdf963c87b5314cfd4e8d', '5c5c15208906156c08681e9d',
       '5c5c15438906156c086821a1', '61793f9577cbc36cfd41067b',
       '5fc60587bc584ab164771223', '5f4b6977b6d42a4c58b75fe3',
       '5c5c156b8906156c08682460', '5e255282c5be9c3a7572fef9',
       '61e40572c154156578e1bb44', '629374789547e340ccac9086',
       '5c5c15678906156c08682422', '5c5c151b8906156c08681e3c',
       '5e987ca964c39c1a94cfe1b4', '5c5c15298906156c08681f72',
       '5c5c154f8906156c08682263', '5c5c154b8906156c0868222b',
       '629374a925e10fc31be2f8ef', '5c7bf93c5aa87addbe99cb1d',
       '5d761fd74b98f4b88b3d6ffe', '5c5c15178906156c08681e03',
       '5db7dc45d5a6a184b2b55369', '5c5c15178906156c086

In [89]:
benedict['item_id'] = benedict['item_id'].fillna('unknown')

In [90]:
benedict['ordered_item_id'].unique()

array(['5c5c15c78906156c086828ad', '619b9faf82d9b4febd5b2dd2',
       '5db7dbacb573995ba60133db', '5c5c16088906156c08682b4e',
       '5eae8379c20497805f628dbd', '62039ee63c24aeac34e164e4',
       '5c5c15858906156c0868259f', '5c5c15838906156c0868257f',
       '6293747b7c740139bc276053', '5c5c162e8906156c08682cdc',
       '5c5c15c28906156c0868287d', '5c5c15ff8906156c08682ad6',
       '629374acc8a5842732b3cd05', '5c7bf93c24daa9995f00cebd',
       '5c5c15ae8906156c086827b1', '5db7dc45b573990ad50133e7',
       '5c5c15ad8906156c086827ab', '63a9c3058055d003e4d77ce4',
       '5c5c15c58906156c08682897', '5c5c15c18906156c08682877',
       '641ae28f57d9256c22f07abf', '5c5c15c78906156c086828ab',
       '62937448c8a58481fbb3cc22', '5c5c15858906156c0868259b',
       '5c5c15b28906156c086827d2', '5c5c15878906156c086825bc',
       '641ad9217fe127baf9f7c717', '641ad77279d9b635c32c0407',
       '5c5c15ae8906156c086827af', '5c5c15c98906156c086828bf',
       '5c5c15cc8906156c086828d9', '619b9f002245d331855

In [91]:
benedict['ordered_item_id'] = benedict['ordered_item_id'].fillna('unknown')

In [92]:
benedict['item_type'].unique()

array(['item', 'general', nan], dtype=object)

In [93]:
benedict['item_type'] = benedict['item_type'].fillna('unknown')

In [94]:
benedict['item_type'] = benedict['item_type'].astype('category')

In [95]:
benedict['ordered_offer_id'].unique()

array(['6477bb3e77ff6b0907a40459', '6477bb3e77ff6b0907a4045a',
       '6477a2656966d4f3007c3e50', ..., '648c3e1f601195c91f728283',
       '648cbd7e28566f7304d63d9c', '648c2e17b5b9abbd19b7b594'],
      dtype=object)

In [96]:
benedict['ordered_offer_id'] = benedict['ordered_offer_id'].fillna('unknown')

In [97]:
benedict['item_name'].unique()

array(['הפוך גדול', 'מיני פנקייק קינדר.', 'ריפיל ר. קינדר', 'כוס קאווה',
       'אספרסו קצר', "בייקון צ'דר פנקייק", 'ב. סלט ירוק',
       'שקשוקה לפי הספר', 'ללא תוספות', 'ב. סלט שוק', 'אגז בנדיקט בייקון',
       'ללא תוספת', "מימוזה פרנץ' 75", 'כרובית קיסר', 'תפוזים',
       'ביצה לבחירה', 'ביצים עלומות לבחירה', 'מימוזה רד', 'בייגל אמריקאי',
       "צ'יפס", 'ריפיל לבן מומס', 'תרד בשמנת', 'בוקר טקסס', 'שוקו קר',
       'פרוסקו קאווה', 'בראנץ בורגר', 'הפוך', 'מימוזה קולדה', "ב. צ'יפס",
       'בומ. גבינת שמנת', 'פנקייק אוכמניות ושוקולד לבן',
       'בומצית בננות חתוכות', 'קרוק מאדאם', 'אבוקדו בייגל ברקפסט',
       'קרעי תפוח אדמה', 'שוקו חם', 'בלאדי מארי', 'מיני פנקייק בננות.',
       'קפה קר', 'אומלט שרימפס', 'אפרול קאריבי', 'בוקר קיסר נייט',
       'בנדיקט סמאש', 'אפרול שפריץ', 'פרללה', 'פנקייק רגיל.',
       '1/3 טובורג', 'תה', 'עין ישראלי', 'ישראלי נייט', 'סלסלת לחמים',
       'מיני פנקייק רגיל', 'בוקר חלומי', 'עלי רוקט',
       'ת. חצילים וגבינת פטה', 'לימונדה', 'סטייק&אגז', 'פילי 

In [98]:
benedict['item_name'] = benedict['item_name'].fillna('unknown')

In [99]:
benedict['offer_name'].unique()

array(['הפוך גדול', 'מיני קינדר', 'רוטב קינדר', 'כוס קאווה', 'אספרסו קצר',
       "בייקון צ'דר פנקייק", 'שקשוקה', 'אגז בנדיקט בייקון',
       "מימוזה פרנץ' 75", 'כרובית קיסר', 'תפוזים', 'ביצה לבחירה',
       'מימוזה רד', 'בייגל אמריקאי', "צ'יפס", 'רוטב לבן מומס',
       'תרד בשמנת', 'בוקר טקסס', 'שוקו קר', 'פרוסקו קאווה',
       'בראנץ בורגר נייט', 'הפוך', 'מימוזה קולדה', 'בראנץ בורגר',
       'גבינת שמנת', 'פנקייק אוכמניות גדול', 'קרוק מאדאם נייט',
       'אבוקדו בייגל ברקפסט נייט', 'קרעי תפוח אדמה', 'שוקו חם',
       'בלאדי מארי', 'מיני בננות.', 'קפה קר', 'אומלט שרימפס',
       'אפרול קאריבי', 'בוקר קיסר נייט', 'בנדיקט סמאש', 'אפרול שפריץ',
       'פרללה', 'פנקייק רגיל', 'טובורג', 'תה', 'ישראלי נייט',
       'סלסלת לחמים', 'ללא תוספות', 'בוקר חלומי',
       'אגז בנדיקט בייקון נייט', 'לימונדה', 'סטייק&אגז',
       'פילי סטייל נייט', 'זירו', 'מיני בייגלה וקרמל מלוח.', 'תפוח',
       'אבוקדו טוסט', 'לקט פטריות', 'חצי לימון', 'בריוש ממולא שוקולד',
       'אמריקנו', 'אבוקדו בייגל', 'קרוק 

In [100]:
benedict['offer_name'] = benedict['offer_name'].replace('', 'none').fillna('none')

In [101]:
benedict['is_item_modifier'].unique()

array([ 0.,  1., nan])

In [102]:
benedict['is_item_modifier'] = benedict['is_item_modifier'].fillna('unknown')

In [103]:
benedict['is_prepared'].unique()

array([ 1.,  0., nan])

In [104]:
benedict['is_prepared'] = benedict['is_prepared'].fillna('unknown')

In [105]:
benedict['is_returned'].unique()

array([ 0.,  1., nan])

In [106]:
benedict['is_returned'] = benedict['is_returned'].fillna('unknown')

In [107]:
benedict['is_cancelled'].unique()

array([ 0.,  1., nan])

In [108]:
benedict['is_cancelled'] = benedict['is_cancelled'].fillna('unknown')

In [109]:
benedict['is_sold'].unique()

array([ 1.,  0., nan])

In [110]:
benedict['is_sold'] = benedict['is_sold'].fillna('unknown')

In [111]:
benedict['is_credit'].unique()

array([ 0.,  1., nan])

In [112]:
benedict['is_credit'] = benedict['is_credit'].fillna('unknown')

In [113]:
benedict['modifiers_sold'].unique()

array([ 0.,  1., nan])

In [114]:
benedict['modifiers_sold'] = benedict['modifiers_sold'].fillna('unknown')

In [115]:
benedict['modifiers_prepared'].unique()

array([ 0.,  1., nan])

In [116]:
benedict['modifiers_prepared'] = benedict['modifiers_prepared'].fillna('unknown')

In [117]:
benedict['total_sales_with_vat'].unique()

array([15.  ,  0.  , 41.  , ..., 14.77, 62.66, 15.26])

In [118]:
benedict['total_sales_with_vat'] = benedict['total_sales_with_vat'].replace('', 'none').fillna('none')

In [119]:
benedict['total_sales_without_vat'].unique()

array([12.82,  0.  , 35.04, ..., 12.62, 53.56, 13.04])

In [120]:
benedict['total_sales_without_vat'] = benedict['total_sales_without_vat'].replace('', 'none').fillna('none')

In [121]:
benedict['operator_amount'].unique()

['positiveSales', 'negativeSales', 'unknown']
Categories (3, object): ['negativeSales', 'positiveSales', 'unknown']

In [122]:
benedict['operator_amount'] = benedict['operator_amount'].fillna('unknown')

In [123]:
benedict['menu_section'].unique()

array(['שתיה', 'אוכל', 'Online', 'המכולת של בנדיקט', 'עובדים', nan,
       'Bakery'], dtype=object)

In [124]:
benedict['menu_section'] = benedict['menu_section'].fillna('unknown')

In [125]:
benedict['menu_subcategory'].unique()

array(['שתיה חמה', 'פנקייקס', 'תוספות', 'מימוזה', 'קלאסיקות',
       'אגז בנדיקט', 'מנות ראשונות', 'מיצים סחוטים', 'צלחות קטנות',
       'מאפים', 'Grill&Eggs', 'שתיה קלה', 'מבעבעים', 'Breakfast in Bread',
       'ביייגלים', 'קוקטיילים', 'אומלטים', 'בוקר מהשדה',
       'שתיה מוגזת/בקבוק', 'בירות', 'בוקר ישראלי/טבעוני', 'פריטי אונליין',
       'בוקר מתוק', 'הערות לשתיה', 'יינות', 'מעדניה', 'אג בולז',
       'ספיישלים', 'שתיה חריפה', 'אוכל עובדים', 'הערות', 'שתיה עובדים',
       'ילדים', 'הרצליה אונליין', nan, 'עוגות', 'מועדון',
       'bundel section', 'אלכוהול', 'default section', 'סוגי ביצים',
       'תוספות לשתיה', 'ביסים', 'עוגיות', 'מלוחים', 'כללי', 'כריכים'],
      dtype=object)

In [126]:
benedict['menu_subcategory'] = benedict['menu_subcategory'].fillna('unknown')

In [127]:
benedict['menu_subcategory'] = benedict['menu_subcategory'].astype('category')

In [128]:
benedict['menu_secondary_subcategory'].unique()

array(['הפוך גדול', 'מיני קינדר', 'רוטב קינדר', 'כוס קאווה', 'אספרסו קצר',
       "בייקון צ'דר פנקייק", 'שקשוקה', 'אגז בנדיקט בייקון',
       "מימוזה פרנץ' 75", 'כרובית קיסר', 'תפוזים', 'ביצה לבחירה',
       'מימוזה רד', 'בייגל אמריקאי', "צ'יפס", 'רוטב לבן מומס',
       'תרד בשמנת', 'בוקר טקסס', 'שוקו קר', 'פרוסקו קאווה',
       'בראנץ בורגר נייט', 'הפוך', 'מימוזה קולדה', 'בראנץ בורגר',
       'גבינת שמנת', 'פנקייק אוכמניות גדול', 'קרוק מאדאם נייט',
       'אבוקדו בייגל ברקפסט נייט', 'קרעי תפוח אדמה', 'שוקו חם',
       'בלאדי מארי', 'מיני בננות.', 'קפה קר', 'אומלט שרימפס',
       'אפרול קאריבי', 'בוקר קיסר נייט', 'בנדיקט סמאש', 'אפרול שפריץ',
       'פרללה', 'פנקייק רגיל', 'טובורג', 'תה', 'ישראלי נייט',
       'סלסלת לחמים', 'ללא תוספות', 'בוקר חלומי',
       'אגז בנדיקט בייקון נייט', 'לימונדה', 'סטייק&אגז',
       'פילי סטייל נייט', 'זירו', 'מיני בייגלה וקרמל מלוח.', 'תפוח',
       'אבוקדו טוסט', 'לקט פטריות', 'חצי לימון', 'בריוש ממולא שוקולד',
       'אמריקנו', 'אבוקדו בייגל', 'קרוק 

In [129]:
benedict['menu_secondary_subcategory'] = benedict['menu_secondary_subcategory'].fillna('unknown')

In [130]:
benedict['menu_secondary_subcategory'] = benedict['menu_secondary_subcategory'].astype('category')

In [131]:
benedict['transaction_date'] = pd.to_datetime(benedict['transaction_date'], errors='coerce', dayfirst=True)

In [132]:
benedict['order_open_time'].describe()

count                  112181
unique                   9611
top       16/06/2023 00:27:05
freq                      305
Name: order_open_time, dtype: object

In [133]:
benedict['order_open_time'] = benedict['order_open_time'].str.strip()

In [134]:
benedict['order_close_time'] = benedict['order_close_time'].str.strip()

In [135]:
benedict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112181 entries, 0 to 112180
Data columns (total 52 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   transaction_id              112181 non-null  object        
 1   transaction_date            112181 non-null  datetime64[ns]
 2   order_open_time             112181 non-null  object        
 3   order_close_time            112181 non-null  object        
 4   is_loyalty_member           112181 non-null  int64         
 5   employee_id                 112181 non-null  object        
 6   employee_name               112181 non-null  object        
 7   order_source                112181 non-null  object        
 8   order_type                  112181 non-null  object        
 9   number_of_diners            112181 non-null  float64       
 10  vat_percentage              112181 non-null  int64         
 11  workflow_profile_name       112181 non-

In [136]:
# Convert the 'date_column' to datetime with the specified format
benedict['order_open_time'] = pd.to_datetime(benedict['order_open_time'], format='%d/%m/%Y %H:%M:%S', errors='coerce', dayfirst=True)

In [137]:
# Convert the 'date_column' to datetime with the specified format
benedict['order_close_time'] = pd.to_datetime(benedict['order_close_time'], format='%d/%m/%Y %H:%M:%S', errors='coerce', dayfirst=True)

In [138]:
benedict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112181 entries, 0 to 112180
Data columns (total 52 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   transaction_id              112181 non-null  object        
 1   transaction_date            112181 non-null  datetime64[ns]
 2   order_open_time             112181 non-null  datetime64[ns]
 3   order_close_time            112181 non-null  datetime64[ns]
 4   is_loyalty_member           112181 non-null  int64         
 5   employee_id                 112181 non-null  object        
 6   employee_name               112181 non-null  object        
 7   order_source                112181 non-null  object        
 8   order_type                  112181 non-null  object        
 9   number_of_diners            112181 non-null  float64       
 10  vat_percentage              112181 non-null  int64         
 11  workflow_profile_name       112181 non-

In [139]:
# check how optimized the dataframe became.
benedict.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112181 entries, 0 to 112180
Data columns (total 52 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   transaction_id              112181 non-null  object        
 1   transaction_date            112181 non-null  datetime64[ns]
 2   order_open_time             112181 non-null  datetime64[ns]
 3   order_close_time            112181 non-null  datetime64[ns]
 4   is_loyalty_member           112181 non-null  int64         
 5   employee_id                 112181 non-null  object        
 6   employee_name               112181 non-null  object        
 7   order_source                112181 non-null  object        
 8   order_type                  112181 non-null  object        
 9   number_of_diners            112181 non-null  float64       
 10  vat_percentage              112181 non-null  int64         
 11  workflow_profile_name       112181 non-

In [140]:
# Save the new dataset to a CSV file
benedict.to_csv('new_benedict.csv', index=False)

In [141]:
benedict.to_excel('new_benedict.xlsx', index=False)

***

In [142]:
# Step 1: Load and preprocess your CSV data
benedict_data = pd.read_csv('new_benedict.csv')

/var/folders/qk/q0fv26q13m760q3ch0fljs240000gn/T/ipykernel_18061/734559934.py:2: DtypeWarning: Columns (21,23,24,25,26,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  benedict_data = pd.read_csv('new_benedict.csv')


In [143]:
# Convert DataFrame to documents with better formatting
def create_text_from_row(row):
    return "Transaction: " + " | ".join([f"{col}: {str(val)}" for col, val in row.items()])

In [144]:
documents = []
for idx, row in benedict_data.iterrows():
    text = create_text_from_row(row)
    metadata = {"source": f"row_{idx}"}
    documents.append(Document(page_content=text, metadata=metadata))

In [145]:
# Create embeddings and vector store
embeddings = FakeEmbeddings(size=768)
vectorstore = FAISS.from_documents(documents, embeddings)

In [146]:
# Initialize GPT4All with correct parameters
callbacks = [StreamingStdOutCallbackHandler()]
model_path = "/Users/cesarchaparro/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin"
llm = GPT4All(
    model=model_path,
    callbacks=callbacks,
    verbose=True
)

Found model file at  /Users/cesarchaparro/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin


objc[18061]: Class GGMLMetalClass is implemented in both /Users/cesarchaparro/opt/anaconda3/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libreplit-mainline-metal.dylib (0x192ee0208) and /Users/cesarchaparro/opt/anaconda3/lib/python3.11/site-packages/gpt4all/llmodel_DO_NOT_MODIFY/build/libllamamodel-mainline-metal.dylib (0x193048208). One of the two will be used. Which one is undefined.


gptj_model_load: loading model from '/Users/cesarchaparro/.cache/gpt4all/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [147]:
# Create QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    verbose=True
)

In [148]:
def chatbot(query):
    try:
        if not query.strip():
            return "Please enter a question."
            
        query_lower = query.lower()

        # Financial Analysis
        if any(word in query_lower for word in ['sales', 'revenue', 'amount', 'financial']):
            if 'total' in query_lower:
                total_sales = benedict_data['total_amount'].sum()
                total_net = benedict_data['net_amount'].sum()
                total_vat = benedict_data['vat_amount'].sum()
                
                return f"""Financial Summary:
                Total Sales: ${total_sales:,.2f}
                Net Amount: ${total_net:,.2f}
                VAT Amount: ${total_vat:,.2f}"""

        # Sales Channels Analysis
        if any(word in query_lower for word in ['channel', 'source', 'order source']):
            channel_summary = benedict_data.groupby('order_source')['total_amount'].agg([
                ('total_sales', 'sum'),
                ('transaction_count', 'count')
            ]).reset_index()
            
            response = "Sales by Channel:\n"
            for _, row in channel_summary.iterrows():
                response += f"{row['order_source']}:\n"
                response += f"- Sales: ${row['total_sales']:,.2f}\n"
                response += f"- Transactions: {row['transaction_count']:,}\n"
            return response

        # Customer Analysis
        if 'customer' in query_lower:
            total_customers = benedict_data['number_of_diners'].sum()
            avg_per_transaction = benedict_data['number_of_diners'].mean()
            loyalty_customers = benedict_data[benedict_data['is_loyalty_member'] == 1]['transaction_id'].count()
            
            return f"""Customer Metrics:
            Total Customers Served: {total_customers:,.0f}
            Average Party Size: {avg_per_transaction:.2f}
            Loyalty Member Transactions: {loyalty_customers:,}"""

        # Time Analysis
        if any(word in query_lower for word in ['time', 'hour', 'period']):
            benedict_data['hour'] = benedict_data['order_open_time'].dt.hour
            hourly_orders = benedict_data.groupby('hour')['transaction_id'].count()
            peak_hour = hourly_orders.idxmax()
            
            return f"""Time Analysis:
            Peak Hour: {peak_hour}:00
            Orders during peak hour: {hourly_orders[peak_hour]:,}
            Average order duration: {(benedict_data['order_close_time'] - benedict_data['order_open_time']).mean().total_seconds() / 60:.1f} minutes"""

        # Menu Analysis
        if any(word in query_lower for word in ['menu', 'item', 'dish']):
            top_items = benedict_data.groupby('item_name')['total_amount'].agg([
                ('total_sales', 'sum'),
                ('order_count', 'count')
            ]).sort_values('total_sales', ascending=False).head(5)
            
            response = "Top 5 Menu Items by Sales:\n"
            for item_name, row in top_items.iterrows():
                response += f"{item_name}:\n"
                response += f"- Sales: ${row['total_sales']:,.2f}\n"
                response += f"- Orders: {row['order_count']:,}\n"
            return response

        # Employee Performance
        if any(word in query_lower for word in ['employee', 'staff', 'server']):
            employee_stats = benedict_data.groupby('employee_name').agg({
                'total_amount': 'sum',
                'transaction_id': 'count',
                'number_of_diners': 'sum'
            }).sort_values('total_amount', ascending=False)
            
            response = "Employee Performance Summary:\n"
            for emp_name, row in employee_stats.head(5).iterrows():
                response += f"{emp_name}:\n"
                response += f"- Sales: ${row['total_amount']:,.2f}\n"
                response += f"- Transactions: {row['transaction_id']:,}\n"
                response += f"- Customers Served: {row['number_of_diners']:,.0f}\n"
            return response

        # Default response with available analysis options
        return """I can help you analyze:
        1. Financial Metrics (sales, revenue, VAT)
        2. Sales Channels Performance
        3. Customer Metrics and Loyalty
        4. Time-based Analysis and Peak Hours
        5. Menu Item Performance
        6. Employee Performance

        Try asking questions like:
        - "Show me the sales by channel"
        - "What are our top menu items?"
        - "Show me customer metrics"
        - "What are our peak hours?"
        - "Show me employee performance"
        - "What are our total sales?"
        """

    except Exception as e:
        return f"Error processing your question: {str(e)}\n\n" + \
               "Please try asking in a different way using the suggested formats."

In [149]:
def chatbot(query):
    try:
        if not query.strip():
            return "Please enter a question."
            
        query_lower = query.lower()

        # VAT Analysis
        if 'without vat' in query_lower:
            # Convert 'total_sales_without_vat' from object to float
            total_sales_without_vat = pd.to_numeric(benedict_data['total_sales_without_vat'], errors='coerce').sum()
            return "Total Sales (excluding VAT): " + str(round(total_sales_without_vat, 2))

        if 'with vat' in query_lower:
            # Convert 'total_sales_with_vat' from object to float
            total_sales_with_vat = pd.to_numeric(benedict_data['total_sales_with_vat'], errors='coerce').sum()
            return "Total Sales (including VAT): " + str(round(total_sales_with_vat, 2))

        # Financial Analysis
        if any(word in query_lower for word in ['sales', 'revenue', 'amount', 'financial']):
            # These columns are already float64
            total_amount = benedict_data['total_amount'].sum()
            net_amount = benedict_data['net_amount'].sum()
            vat_amount = benedict_data['vat_amount'].sum()
            gross_amount = benedict_data['gross_amount'].sum()
            
            response = "Financial Summary:\n"
            response += f"Total Amount: {total_amount:.2f}\n"
            response += f"Net Amount: {net_amount:.2f}\n"
            response += f"VAT Amount: {vat_amount:.2f}\n"
            response += f"Gross Amount: {gross_amount:.2f}"
            return response

        # Refunds Analysis
        if 'refund' in query_lower:
            # These columns are float64
            refund_with_vat = benedict_data['refund_with_vat'].sum()
            refund_without_vat = benedict_data['refund_without_vat'].sum()
            refund_vat_amount = benedict_data['refund_vat_amount'].sum()
            
            response = "Refund Summary:\n"
            response += f"Refunds (with VAT): {refund_with_vat:.2f}\n"
            response += f"Refunds (without VAT): {refund_without_vat:.2f}\n"
            response += f"Refund VAT Amount: {refund_vat_amount:.2f}"
            return response

        # Daily Analysis
        if 'daily' in query_lower:
            # Group by date (transaction_date is datetime64[ns])
            daily_summary = benedict_data.groupby(benedict_data['transaction_date'].dt.date).agg({
                'total_amount': 'sum',
                'net_amount': 'sum',
                'vat_amount': 'sum'
            })
            
            response = "Daily Summary (Last 5 days):\n"
            for date, row in daily_summary.tail(5).iterrows():
                response += f"\nDate: {date}\n"
                response += f"Total Amount: {row['total_amount']:.2f}\n"
                response += f"Net Amount: {row['net_amount']:.2f}\n"
                response += f"VAT Amount: {row['vat_amount']:.2f}"
            return response

        # Price Analysis
        if 'price' in query_lower:
            # These columns are float64
            base_price_avg = benedict_data['base_price'].mean()
            final_price_avg = benedict_data['final_item_price'].mean()
            discount_avg = benedict_data['discount_amount'].mean()
            
            response = "Price Analysis:\n"
            response += f"Average Base Price: {base_price_avg:.2f}\n"
            response += f"Average Final Price: {final_price_avg:.2f}\n"
            response += f"Average Discount: {discount_avg:.2f}"
            return response

        # Cancelled and Returned Analysis
        if any(word in query_lower for word in ['cancel', 'return']):
            # These columns are float64
            cancelled_amount = benedict_data['cancelled_amount'].sum()
            returned_amount = benedict_data['returned_amount'].sum()
            
            response = "Cancelled and Returned Summary:\n"
            response += f"Cancelled Amount: {cancelled_amount:.2f}\n"
            response += f"Returned Amount: {returned_amount:.2f}"
            return response

        # Default response
        return """I can help you analyze financial data:
        Try asking questions like:
        - "What is the total sales without VAT?"
        - "What is the total sales with VAT?"
        - "Show me financial summary"
        - "Show me refund analysis"
        - "Show me daily summary"
        - "Show me price analysis"
        - "Show me cancelled and returned amounts"
        """

    except Exception as e:
        return "Error processing your question: " + str(e) + "\n\n" + \
               "Please try asking in a different way using the suggested formats."

In [150]:
def chatbot(query):
    try:
        if not query.strip():
            return "Please enter a question."
            
        query_lower = query.lower()

        # Order Duration Analysis
        if any(word in query_lower for word in ['duration', 'time', 'long', 'short']):
            # First, let's check the data types
            benedict_data['order_open_time'] = pd.to_datetime(benedict_data['order_open_time'], format='%Y-%m-%d %H:%M:%S')
            benedict_data['order_close_time'] = pd.to_datetime(benedict_data['order_close_time'], format='%Y-%m-%d %H:%M:%S')
            
            # Calculate time difference in minutes
            time_diff = benedict_data['order_close_time'].sub(benedict_data['order_open_time'])
            duration_minutes = time_diff.dt.total_seconds() / 60
            
            # Calculate basic statistics
            avg_duration = duration_minutes.mean()
            min_duration = duration_minutes.min()
            max_duration = duration_minutes.max()
            
            return (f"Order Duration Analysis:\n"
                   f"Average Duration: {avg_duration:.1f} minutes\n"
                   f"Shortest Order: {min_duration:.1f} minutes\n"
                   f"Longest Order: {max_duration:.1f} minutes")

        # Default response
        return """I can help you analyze order durations.
        Try asking questions like:
        - "What is the average order duration?"
        - "Show me order times"
        """

    except Exception as e:
        # Add debugging information
        print("Error details:", str(e))
        print("Open time dtype:", benedict_data['order_open_time'].dtype)
        print("Close time dtype:", benedict_data['order_close_time'].dtype)
        return "Error processing your question. Please try again."

In [151]:
def chatbot(query):
    try:
        if not query.strip():
            return "Please enter a question."
            
        query_lower = query.lower()

        # Top dishes by table size analysis
        if any(word in query_lower for word in ['table', 'dishes', 'diners']):
            # Group by number of diners and item name, count occurrences and sum amounts
            table_dishes = benedict_data.groupby(['number_of_diners', 'item_name']).agg({
                'transaction_id': 'count',  # Count of orders
                'total_amount': 'sum'      # Total sales amount
            }).reset_index()
            
            # Get top 3 dishes for each table size
            top_dishes = (table_dishes.sort_values(['number_of_diners', 'transaction_id'], 
                                                 ascending=[True, False])
                         .groupby('number_of_diners')
                         .head(3))
            
            # Format the response
            response = "Top 3 Dishes by Table Size:\n"
            
            for table_size in sorted(top_dishes['number_of_diners'].unique()):
                response += f"\nTable Size: {int(table_size)} diners\n"
                table_data = top_dishes[top_dishes['number_of_diners'] == table_size]
                
                for _, row in table_data.iterrows():
                    response += f"- {row['item_name']}\n"
                    response += f"  Orders: {int(row['transaction_id'])}\n"
                    response += f"  Total Sales: ${row['total_amount']:,.2f}\n"
            
            return response

        # Default response
        return """I can help you analyze top dishes by table size.
        Try asking questions like:
        - "What are the top dishes by table size?"
        - "Show me popular dishes for each table size"
        - "What do different size groups order?"
        """

    except Exception as e:
        return f"Error processing your question: {str(e)}\nPlease try asking in a different way."

In [152]:
# Create Gradio interface
demo = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(
        lines=2,
        placeholder="Enter your question here...",
        label="Question"
    ),
    outputs=gr.Textbox(
        label="Answer",
        lines=10
    ),
    title="Restaurant Data Assistant",
    description="Ask questions about your restaurant data (e.g., 'What were the first 5 transactions?')"
)
# Launch the interface
if __name__ == "__main__":
    demo.launch(share=False)

IMPORTANT: You are using gradio version 3.32.0, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
